In [1]:
import pandas as pd
import numpy as np

# Scraping
from chembl_webresource_client.new_client import new_client



There are validated (?) SMILES for all compounds in AqSolDB, there only missing structural info is the HLC compounds that are not in AqSolDB ---> 2226 compounds

In [148]:
aqsol = pd.read_csv('../AqSolDB/aqsoldb_clean_1115.csv')
henry = pd.read_csv('../HLCDB/henry_shrinked.csv')
species = pd.read_csv('../HLCDB/species_excelled.csv').drop(columns=['Unnamed: 7'])
species = species.loc[species.inchikey != 'inchikey']

In [149]:
unmatched_hlc = species.loc[~species['inchikey'].isin(aqsol['inchikey'])].copy(deep=True) # inchikey in hlc but not aqsoldb
unmatched_hlc

,id,iupac,formula,trivial,casrn,inchikey,subcat_id
1,2,ozone,O<sub>3</sub>,NaN,10028-15-6,CBENFWSGALASAD-UHFFFAOYSA-N,1
2,3,hydrogen atom,H,NaN,12385-13-6,YZCKVEUIGOORGS-UHFFFAOYSA-N,2
3,4,hydrogen,H<sub>2</sub>,NaN,1333-74-0,UFHFLCQGNIYNRP-UHFFFAOYSA-N,2
4,5,deuterium,D<sub>2</sub>,NaN,7782-39-0,UFHFLCQGNIYNRP-VVKOMZTBSA-N,2
5,6,hydroxyl radical,OH,NaN,3352-57-6,TUJKJAMUKRIRHC-UHFFFAOYSA-N,2
...,...,...,...,...,...,...,...
4626,4627,iodophenylmercury,C<sub>6</sub>H<sub>5</sub>HgI,NaN,823-04-1,BISBXZWWFIOZSX-UHFFFAOYSA-M,60
4627,4628,tetramethyl lead,C<sub>4</sub>H<sub>12</sub>Pb,NaN,75-74-1,XOOGZRUBTYCLHG-UHFFFAOYSA-N,61
4628,4629,ethyltrimethylplumbane,C<sub>5</sub>H<sub>14</sub>Pb,NaN,1762-26-1,KHQJREYATBQBHY-UHFFFAOYSA-N,61
4629,4630,diethyldimethylplumbane,C<sub>6</sub>H<sub>16</sub>Pb,diethyldimethyl lead,1762-27-2,OLOAJSHVLXNSQV-UHFFFAOYSA-N,61


In [4]:
unmatched_hlc['inchikey']

1       CBENFWSGALASAD-UHFFFAOYSA-N
2       YZCKVEUIGOORGS-UHFFFAOYSA-N
3       UFHFLCQGNIYNRP-UHFFFAOYSA-N
4       UFHFLCQGNIYNRP-VVKOMZTBSA-N
5       TUJKJAMUKRIRHC-UHFFFAOYSA-N
                   ...             
4626    BISBXZWWFIOZSX-UHFFFAOYSA-M
4627    XOOGZRUBTYCLHG-UHFFFAOYSA-N
4628    KHQJREYATBQBHY-UHFFFAOYSA-N
4629    OLOAJSHVLXNSQV-UHFFFAOYSA-N
4630    KGFRUGHBHNUHOS-UHFFFAOYSA-N
Name: inchikey, Length: 2226, dtype: object

# CheMBL

In [6]:
# molecule = new_client.molecule
# mol = molecule.filter(molecule_structures__standard_inchi_key='BSYNRYMUTXBXSQ-UHFFFAOYSA-N').only([
#     'molecule_chembl_id', 'pref_name', 'molecule_structures'])
# mol = mol[0]
# mol.keys()
# mol.get('molecule_chembl_id')

# print(mol.get('molecule_structures').keys())
# print(mol.get('molecule_structures').get('canonical_smiles'))
# print(mol.get('molecule_structures').get('standard_inchi'))

In [8]:
def filter(inchikey:str):
    molecule = new_client.molecule
    mol = molecule.filter(molecule_structures__standard_inchi_key=inchikey).only([
        'molecule_chembl_id', 'pref_name', 'molecule_structures'])
    return mol[0]

In [9]:
ft = unmatched_hlc['inchikey'].apply(lambda x: filter(x)).rename('request') # .get('molecule_structures').get('canonical_smiles'))

In [22]:
ft['request'].to_csv('smiles_chembl.csv')

In [25]:
chembl = pd.read_csv('smiles_chembl.csv',index_col=0)
chembl

,request
1,"{'molecule_chembl_id': 'CHEMBL2447938', 'molec..."
2,NaN
3,NaN
4,NaN
5,"{'molecule_chembl_id': 'CHEMBL1689064', 'molec..."
...,...
4626,NaN
4627,NaN
4628,NaN
4629,NaN


In [114]:
dicts = ft.request.dropna().apply(lambda x: dict(x))
# dicts = chembl.request.d\ropna().apply(lambda x: dict(x))
dicts

1       {'molecule_chembl_id': 'CHEMBL2447938', 'molec...
5       {'molecule_chembl_id': 'CHEMBL1689064', 'molec...
10      {'molecule_chembl_id': 'CHEMBL186537', 'molecu...
13      {'molecule_chembl_id': 'CHEMBL1200689', 'molec...
20      {'molecule_chembl_id': 'CHEMBL1161681', 'molec...
                              ...                        
4563    {'molecule_chembl_id': 'CHEMBL2251448', 'molec...
4566    {'molecule_chembl_id': 'CHEMBL4238997', 'molec...
4569    {'molecule_chembl_id': 'CHEMBL3183859', 'molec...
4593    {'molecule_chembl_id': 'CHEMBL1411246', 'molec...
4599    {'molecule_chembl_id': 'CHEMBL1474517', 'molec...
Name: request, Length: 737, dtype: object

In [115]:
chembl_ids = []
canonical_smiles = []
inchis = []
chembl_inchikeys = []

for k, req in enumerate(dicts):
    # id = 
    id = req.get('molecule_chembl_id')
    chembl_ids.append(id)
    sm = req.get('molecule_structures').get('canonical_smiles')
    canonical_smiles.append(sm)
    inch = req.get('molecule_structures').get('standard_inchi')
    inchis.append(inch)
    key = req.get('molecule_structures').get('standard_inchi_key')
    chembl_inchikeys.append(key)

# dicts[1].get('molecule_structures').get('canonical_smiles')

In [116]:
dicts = pd.DataFrame(dicts)

dicts['chembl_id'] = chembl_ids
dicts['smiles'] = canonical_smiles
dicts['inchi'] = inchis
dicts['chembl_inchikeys'] = chembl_inchikeys

In [117]:
dicts

,request,chembl_id,smiles,inchi,chembl_inchikeys
1,"{'molecule_chembl_id': 'CHEMBL2447938', 'molec...",CHEMBL2447938,O=[O+][O-],InChI=1S/O3/c1-3-2,CBENFWSGALASAD-UHFFFAOYSA-N
5,"{'molecule_chembl_id': 'CHEMBL1689064', 'molec...",CHEMBL1689064,[OH],InChI=1S/HO/h1H,TUJKJAMUKRIRHC-UHFFFAOYSA-N
10,"{'molecule_chembl_id': 'CHEMBL186537', 'molecu...",CHEMBL186537,[N-]=[N+]=N,InChI=1S/HN3/c1-3-2/h1H,JUINSXZKUKVTMD-UHFFFAOYSA-N
13,"{'molecule_chembl_id': 'CHEMBL1200689', 'molec...",CHEMBL1200689,[N]=O,InChI=1S/NO/c1-2,MWUXSHHQAYIFBG-UHFFFAOYSA-N
20,"{'molecule_chembl_id': 'CHEMBL1161681', 'molec...",CHEMBL1161681,O=NO,"InChI=1S/HNO2/c2-1-3/h(H,2,3)",IOVCWXUNBOPUCH-UHFFFAOYSA-N
...,...,...,...,...,...
4563,"{'molecule_chembl_id': 'CHEMBL2251448', 'molec...",CHEMBL2251448,CCO[PH](=O)[O-].CCO[PH](=O)[O-].CCO[PH](=O)[O-...,"InChI=1S/3C2H7O3P.Al/c3*1-2-5-6(3)4;/h3*6H,2H2...",ZKZMJOFIHHZSRW-UHFFFAOYSA-K
4566,"{'molecule_chembl_id': 'CHEMBL4238997', 'molec...",CHEMBL4238997,C[Si](C)(C)O,"InChI=1S/C3H10OSi/c1-5(2,3)4/h4H,1-3H3",AAPLIUHOKVUFCC-UHFFFAOYSA-N
4569,"{'molecule_chembl_id': 'CHEMBL3183859', 'molec...",CHEMBL3183859,CO[Si](OC)(OC)OC,"InChI=1S/C4H12O4Si/c1-5-9(6-2,7-3)8-4/h1-4H3",LFQCEHFDDXELDD-UHFFFAOYSA-N
4593,"{'molecule_chembl_id': 'CHEMBL1411246', 'molec...",CHEMBL1411246,S=C([S-])NCCNC(=S)[S-].[Zn+2],InChI=1S/C4H8N2S4.Zn/c7-3(8)5-1-2-6-4(9)10;/h1...,AMHNZOICSMBGDH-UHFFFAOYSA-L


In [119]:
dicts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 1 to 4599
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   request           737 non-null    object
 1   chembl_id         737 non-null    object
 2   smiles            737 non-null    object
 3   inchi             737 non-null    object
 4   chembl_inchikeys  737 non-null    object
dtypes: object(5)
memory usage: 34.5+ KB


In [118]:
# dicts.drop('request',axis=1).to_csv('chembl_scrapes.csv')

# need to add back inchikeys from hlc/species -> compare

In [123]:
# unmatched_hlc.loc[unmatched_hlc.id.isin(dicts.index)]

,id,iupac,formula,trivial,casrn,inchikey,subcat_id
4,5,deuterium,D<sub>2</sub>,NaN,7782-39-0,UFHFLCQGNIYNRP-VVKOMZTBSA-N,2
20,21,nitrous acid,HNO<sub>2</sub>,NaN,7782-77-6,IOVCWXUNBOPUCH-UHFFFAOYSA-N,3
29,30,chlorine atom,Cl,NaN,22537-15-1,ZAMOUSCENKQFHK-UHFFFAOYSA-N,5
30,31,hydrogen chloride,HCl,NaN,7647-01-0,VEXZGXHMUGYJMC-UHFFFAOYSA-N,5
31,32,hypochlorous acid,HOCl,NaN,7790-92-3,QWPPOHNGKGFGJK-UHFFFAOYSA-N,5
...,...,...,...,...,...,...,...
4517,4517,propetamphos,C<sub>10</sub>H<sub>20</sub>NO<sub>4</sub>PS,NaN,31218-83-4,BZNDWPRGXNILMS-VQHVLOKHSA-N,52
4539,4539,chlorthiophos,C<sub>11</sub>H<sub>15</sub>Cl<sub>2</sub>O<su...,NaN,21923-23-9,JAZJVWLGNLCNDD-UHFFFAOYSA-N,52
4567,4566,silicic acid,Si(OH)<sub>4</sub>,NaN,10193-36-9,RMAQACBXLXPBSY-UHFFFAOYSA-N,55
4570,4569,pentamethyldisiloxanol,C<sub>5</sub>H<sub>16</sub>O<sub>2</sub>Si<sub...,NaN,56428-93-4,FGOLZCPMTWJPOU-UHFFFAOYSA-N,55


# PubChem

In [124]:
import requests

In [126]:
key = 'UFHFLCQGNIYNRP-VVKOMZTBSA-N'

url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/InChiKey/{','.join(map(str, key))}/property/CanonicalSMILES/JSON"

r = requests.get(url)




In [153]:
unmatched_hlc['inchikey'].to_csv('unmatched_inchikeys.csv',index=None, header=None)

In [157]:
pubchem = pd.read_csv('unmatched_inchikeys w pubchem smiles.csv',sep='')
pubchem

,CBENFWSGALASAD-UHFFFAOYSA-N\t[O-][O+]=O
0,YZCKVEUIGOORGS-UHFFFAOYSA-N\t[H]
1,UFHFLCQGNIYNRP-UHFFFAOYSA-N\t[HH]
2,UFHFLCQGNIYNRP-VVKOMZTBSA-N\t[2H][2H]
3,TUJKJAMUKRIRHC-UHFFFAOYSA-N\t[OH]
4,MHAJPDPJQMAIIY-UHFFFAOYSA-M\tO[O-]
...,...
2078,BISBXZWWFIOZSX-UHFFFAOYSA-M\tC1=CC=[C]C=C1.[I-...
2079,XOOGZRUBTYCLHG-UHFFFAOYSA-N\tC[Pb](C)(C)C
2080,KHQJREYATBQBHY-UHFFFAOYSA-N\tCC[Pb](C)(C)C
2081,OLOAJSHVLXNSQV-UHFFFAOYSA-N\tCC[Pb](C)(C)CC
